In [1]:
# https://www.kaggle.com/alexanderkireev/experiments-with-imbalance-nn-architecture/code


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'
import gc


In [3]:
# data read

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
print('load train....')

train_df = pd.read_csv("train.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

print('load test....')
test_df = pd.read_csv("test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])
len_train = len(train_df)
train_df=train_df.append(test_df)
del test_df; gc.collect()



load train....


KeyboardInterrupt: 

In [ ]:

print('hour, day, wday....')
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['wday']  = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')


In [ ]:
# need more!
print('grouping by ip-day-hour combination....')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
# print(gp.head())

del gp; gc.collect()


In [ ]:
print('group by ip-app combination....')
gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp, on=['ip','app'], how='left')
del gp; gc.collect()

# print(train_df.head())

In [ ]:
print('group by ip-app-os combination....')
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp; gc.collect()
# print(train_df.head())

In [ ]:
print("vars and data type....")
train_df['qty'] = train_df['qty'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

# print(train_df['qty'].max(),train_df['ip_app_count'].max(),train_df['ip_app_os_count'].max())

In [ ]:
from sklearn.preprocessing import LabelEncoder
train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)



In [ ]:
gc.collect()

print ('final part of preparation....')
test_df = train_df[len_train:]
train_df = train_df[:len_train]
# y_train = train_df['is_attributed'].values
y_train = train_df['is_attributed']

train_df.drop(['click_id','click_time','ip','is_attributed'],1,inplace=True)
print(train_df.shape)
print(y_train.shape)


In [4]:
# print("save test")
# test_df.to_csv('test_data_code2.csv')
# print("save train")
# train_df.to_csv('train_data_code2.csv')
print("save y")
y_train = pd.read_csv("train.csv", dtype=dtypes, usecols=['is_attributed'])

y_train = train_df['is_attributed']
y_train.to_csv('y_code2.csv')

# print(train_df.shape)
print(y_train.shape)

save y
(184903890, 1)


In [ ]:
print ('neural network....')
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from keras.callbacks import Callback
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
# simple MY nn
my_init = 'random_uniform'
model = Sequential()
model.add(Dense(32,activation='relu',kernel_initializer=my_init,
                bias_initializer='zeros',input_shape=(train_df.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(64,kernel_initializer=my_init,
                bias_initializer='zeros',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# model.add(Dense(128,kernel_initializer=my_init,
#                 bias_initializer='zeros',activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(64,kernel_initializer=my_init,
#                 bias_initializer='zeros',activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

model.add(Dense(32,kernel_initializer=my_init,
                bias_initializer='zeros',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(16,kernel_initializer=my_init,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:

epochs = 3
batch_size = 10000
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(list(train_df)[0]) / batch_size) * epochs
lr_init, lr_fin = 0.002, 0.0002
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.002, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])
class_weight = {0:.01,1:.99}
model.fit(train_df, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weight, shuffle=True, verbose=2)
del train_df, y_train; gc.collect()

json_string = model.to_json()
with open("imbalanced_model.json", "w") as json_file:
    json_file.write(json_string)
model.save_weights('imbalanced_data.h5')

In [ ]:
sub = pd.DataFrame()
sub['click_id'] = test_df['click_id']
test_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)
test_df = get_keras_data(test_df)


In [ ]:
print("predicting....")
sub['is_attributed'] = model.predict(test_df, batch_size=batch_size, verbose=2)
del test_df; gc.collect()
print("writing....")
sub.to_csv('imbalanced_data.csv',index=False)